In [3]:
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
# Định nghĩa URL cơ sở của website và endpoint cụ thể
# base_url = "https://envisoft.gov.vn/eos" 
base_url = "https://enviinfo.cem.gov.vn"
endpoint = "/eip/default/call/json/get_indicators_have_data"
# endpoint = "/services/call/json/qi_detail"

url = base_url + endpoint

# Tham số yêu cầu (Bạn cần thay thế giá trị này bằng các giá trị thực tế tương ứng với yêu cầu của bạn)
payload = {
    'station_id': 28602553176253587650986727137,  # Thay "YOUR_STATION_ID_HERE" bằng ID trạm thực tế
    'from_public': 1,
    'station_type': 4  # Thay "YOUR_STATION_TYPE_HERE" bằng loại trạm thực tế
}

# Tiến hành yêu cầu POST
response = requests.post(url, data=payload)

# Kiểm tra mã trạng thái của phản hồi
if response.status_code == 200:
    print(response.json())
else:
    print("Lỗi:", response.status_code)


In [ ]:
# Define your base URL and specific endpoints
base_url = "https://enviinfo.cem.gov.vn"
endpoint_aqi_data = "/eip/default/call/json/get_aqi_data%3Fdate%3D1900%26aqi_type%3D0"
endpoint_indicators_data = "/eip/default/call/json/get_indicators_have_data"

# Construct complete URLs for the requests
url_aqi_data = f"{base_url}{endpoint_aqi_data}"
url_indicators_data = f"{base_url}{endpoint_indicators_data}"

# Load province data to get province IDs
with open("data/province_data.json", "r") as file:
    province_data = json.load(file)

    province_ids = [BeautifulSoup(province[1], "html.parser").find("a")["data-id"] for province in province_data['aaData']]
    province_latitudes = [BeautifulSoup(province[1], "html.parser").find("a")["data-lat"] for province in province_data['aaData']]
    province_longitudes = [BeautifulSoup(province[1], "html.parser").find("a")["data-lon"] for province in province_data['aaData']]

# Initialize payloads for the requests
payload_aqi = {
    'sEcho': 1,
    'iColumns': 9,
    'sColumns': ',,,,,,,,',
    'iDisplayStart': 0,
    'iDisplayLength': 1900,
    'mDataProp_0': 0,
    'sSearch_0': '',
    'bRegex_0': False,
    'bSearchable_0': True,
    # Additional parameters will be filled as needed
}

payload_indicators = {
    'station_id': 0,  # Will be updated per province
    'from_public': 1,
    'station_type': 4,
}

# Process each province
for province_id in province_ids[60:77]:
    # Update payloads with the current province ID
    payload_aqi["station_id"] = province_id
    payload_indicators["station_id"] = province_id

    # Fetch available indicators for the current province
    response_indicators = requests.post(url_indicators_data, data=payload_indicators)
    if response_indicators.status_code == 200:
        indicator_html = response_indicators.json().get('html', '')
        soup = BeautifulSoup(indicator_html, 'html.parser')
        selected_indicators = [option.text for option in soup.find_all('option', selected=True)]
        
        # Fetch AQI data for the current province
        response_aqi = requests.post(url_aqi_data, data=payload_aqi)
        if response_aqi.status_code == 200:
            data = response_aqi.json()
            # Assign the new column names including the indicators
            data['aoColumns'] = ['ID', 'Date', 'AQI'] + selected_indicators

            # Save the updated data to a file
            with open(f"data-province/test-{province_id}.json", "w") as file:
                json.dump(data, file)
            # print(f"Data for province {province_id} saved successfully.")
        else:
            print(f"Error fetching AQI data for province {province_id}: {response_aqi.status_code}")
    else:
        print(f"Error fetching indicators for province {province_id}: {response_indicators.status_code}")

In [5]:
# Configuration for the API request
api_base_url = "https://enviinfo.cem.gov.vn"
stations_endpoint = "/eip/default/call/json/get_stations_by_province"
complete_stations_url = f"{api_base_url}{stations_endpoint}"

# Detailed request payload for fetching station data
stations_payload = {
    'sEcho': 1,  # Request identifier for DataTables
    'iColumns': 3,  # Number of columns in the DataTable
    'sColumns': ',,',  # Column identifiers
    'iDisplayStart': 0,  # Start point in the data set (for pagination)
    'iDisplayLength': 77,  # Number of records to fetch
    'mDataProp_0': 0,  # Data property for the first column
    'sSearch_0': '',  # Search term for the first column
    'bRegex_0': False,  # Use regex for the first column search term?
    'bSearchable_0': True,  # Can the first column be searched?
    'mDataProp_1': 1,  # Data property for the second column
    'station_type': 4  # Filter for station type
}

# Perform the POST request to fetch station data by province
station_response = requests.post(complete_stations_url, data=stations_payload)

# Handling the response
if station_response.status_code == 200:
    # Saving the response data to a JSON file
    with open('province-stations/province_station_data.json', 'w') as file:
        json.dump(station_response.json(), file)
    print("Station data saved successfully.")
else:
    print(f"Error fetching station data: {station_response.status_code}")

Station data saved successfully.


In [34]:
# # Open all files in data-provinces-name folder
# # Check if iTotalRecords < 600 then remove the file
# import os

# for filename in os.listdir("data-province"):
#     with open(f"data-province/{filename}", "r") as f:
#         data = json.load(f)
#         # if iTotalRecords < 600 or number of aoColumns < 9
#         if data['iTotalRecords'] < 600 or len(data['aoColumns']) < 9:
#             os.remove(f"data-province/{filename}")
#             print(f"Deleted: {filename}")

Deleted: test-29203727697074312726675247132.json
Deleted: test-29498302480362039106897316897.json
Deleted: test-28505272740301122608933325208.json
Deleted: test-30075912630841951814419146753.json
Deleted: test-28505279528702941738980821400.json
Deleted: test-28916774462801800655608897080.json
Deleted: test-28916613404279292928684613176.json
Deleted: test-29544533341424688633522251105.json
Deleted: test-28602837126985114455002404577.json
Deleted: test-28505268571336961948594948504.json
Deleted: test-29213751141295132066317063859.json
Deleted: test-29251324890023757539631193763.json
Deleted: test-29337259840501128739480820670.json
Deleted: test-28602919547037635858334045921.json
Deleted: test-30075907004585009313561110529.json
Deleted: test-29337245452040751228582255550.json


In [4]:
with open("data/province_station_data.json", "r") as f:
    province_data = json.load(f)
    province_names = []
    province_ids = []
    for province in province_data['aaData']:
        html_snippet = province[1]
        soup = BeautifulSoup(html_snippet, "html.parser")
        province_id = soup.find("a")["data-id"]
        province_name = soup.find("a").get_text()

        province_name = province_name.replace(":", "").replace("/", "-").replace("\\", "-")
        
        # Find the file with their data-id then change to the province name if exists, ifnot, ignore
        try:
            with open(f"data-province/test-{province_id}.json", "r") as file:
                data = json.load(file)
                with open(f"data-provinces-name/{province_name}.json", "w") as new_file:
                    json.dump(data, new_file)
        except FileNotFoundError:
            print(f"File with data-id {province_id} not found.")
        except Exception as e:
            print(f"Error processing data for province {province_name}: {e}")